# Pivot model

This notebook generates simplifications with the KiS-based Pivot model.

These simplifications can then be evaluated in the evaluation.ipynb.

This notebook requires the [KiS repository](https://github.com/tingofurro/keep_it_simple) and the [released KiS model](https://github.com/tingofurro/keep_it_simple/releases/tag/0.1).

### Load the datasets

In [2]:
import pandas as pd

wiki_path = "../data/wiki_eval.csv"

wiki_eval = pd.read_csv(wiki_path)
wiki_eval.head()

,paragraph,size,guts1,guts2,guts3,pivot
0,Erst ab 1860 unter Fürst Adolf Georg wurden di...,medium,Erst ab 1860 unter Fürsten Adolf Georg wurden ...,Schon ab 1860 unter Fürst Ad wurden die Schlos...,NaN,Erst 1860 wurden die Palasträume komplett reno...
1,Im Jahr 2000 fand der österreichische Flieger ...,short,Das Jahr 2000 fand der österreichischen Fliege...,Im Jahr 2000 fand die österreichische Flieger ...,NaN,Im Jahr 2000 fand der österreichische Pilot Si...
2,Der Ort Arbing ist am Fuß der mittelalterliche...,long,Das Ort Arbing ist im 15 Gebäude entstanden un...,Der Ort Arbing am Fuß der mittelalterlichen St...,NaN,Das Dorf Arbing liegt am Fuße des mittelalterl...
3,Die Ortschaften im Verwaltungsbezirk der Bürge...,long,Die Ortschaften im Verwaltungs Bezirk der Bürg...,Die Ortschaften im Verwaltungsgebiet der Bürge...,NaN,"Vor 1794 gehörte Bleialf zu Prüm, dessen Fürst..."
4,"Die Ortschaft ist über die A726, die Strathave...",medium,"Die Ortschaft ist über den A726, über East Kil...","Die Ortschaft ist über A7, die Strathaven mit ...",NaN,Das Dorf ist mit dem Straßennetz über die A726...


In [2]:
import pandas as pd
tc_path = "../data/tc_eval.csv"

tc_eval = pd.read_csv(tc_path)
tc_eval.head()

,paragraph,reference,pivot,guts1,guts2
0,"Wegen dieser leichten Vergänglichkeit wurde ,S...","Weil Seifenblasen nicht lange halten, wurden s...",Aufgrund dieser leichten Vergänglichkeit wurde...,"Das leichte Vergänglichkeit wurde ,Das Blase‘ ...","Mit dieser leichten Vergänglichkeit wird ,Seif..."
1,"Eine Seifenblase entsteht, wenn sich ein dünne...","Eine Seifenblase entsteht, wenn sich eine klei...","Eine Seifenblase entsteht, wenn ein dünner Was...","Die Seifenblase entsteht, sobald sich ein dünn...","Ein Seifenblase entsteht,, wenn sich ein dicke..."
2,Die Schichtdicke der Seifenblase lässt sich be...,Die Dicke der Seifenblase lässt sich beobachte...,Die Schichtdicke der Seifenblase kann beobacht...,Die Schichtdicke ist die Seifenblase lässt sic...,Die Schichtdicke des Seifenblasen lässt sich b...
3,"So schildert der achte Gesang beispielsweise, ...",So schildert der achte Besang beispielsweise f...,"Zum Beispiel beschreibt das achte Lied, wie er...",Der achte Gesang schildert den achte Gesang be...,"Der achte Gesang hingegen, wie im Haus des Pha..."
4,Vergleichbare Stellen belegen eine sorgfältige...,"Es gibt Schriften, die erzählen, dass es üblic...",Ähnliche Orte zeigen eine sorgfältige Reinigun...,Esbare Stellen belegen eine gründliche Reinigu...,Berbare Stellen belegen eine gründliche Reinig...


### Translate paragraphs to english

In [4]:
from transformers import pipeline

# Load de-en translation model 
pipe = pipeline("translation", model='Helsinki-NLP/opus-mt-de-en', device=0, use_fast=True)

In [ ]:
# translate wiki paragraphs
wiki_o_en = [t['translation_text'] for t in pipe(wiki_eval['paragraph'].to_list())]

In [6]:
# translate tc paragraphs
tc_o_en = [t['translation_text'] for t in pipe(tc_eval['paragraph'].to_list())]

In [8]:
tc_o_en[0]

'Because of this slight transience, "soap bubble" became a metaphor for something that, although attractive, is contentless and contentless. In art, at the latest since the Baroque, the soap bubble is consistently used iconographically as a vanitas symbol and reflects both the beauty and the fleeting nature of human life.'

### Simplify the English paragraphs

In [9]:
from model_generator import Generator

# load the KiS generator model
model = Generator("gpt2-medium", max_output_length=230, device='cuda')
model.reload("gpt2_med_keep_it_simple.bin")
model.eval()

<All keys matched successfully>


In [ ]:
# Generate Simplifications for Wiki
model_output = model.generate(wiki_o_en, num_runs=1, sample=False) # Greedy decoding
wiki_simpl_en = [o['output_text'] for o in model_output]

In [10]:
# Generate Simplifications for TC
model_output = model.generate(tc_o_en, num_runs=1, sample=False) # Greedy decoding
tc_simpl_en = [o['output_text'] for o in model_output]

In [11]:
tc_simpl_en[0]

'Because of this slight transience, "soap bubble" became a metaphor for something that, although attractive, is not contentless and does not reflect the beauty of life. In art, the baroque has always used soap bubble as a symbol of vanitas, and reflects both the beauty and the fleeting nature of human life.'

### Translate the simplifications back to German

In [12]:
pipe = pipeline("translation", model='Helsinki-NLP/opus-mt-en-de', device=0, use_fast=True)

In [ ]:
# re-translate simplified wiki paragraphs
wiki_simpl_de = [t['translation_text'] for t in pipe(wiki_simpl_en)]

In [15]:
# re-translate simplified tc paragraphs
tc_simpl_en = [t['translation_text'] for t in pipe(tc_simpl_en)]

In [16]:
tc_simpl_en[0]

'Aufgrund dieser leichten Vergänglichkeit wurde die „Seifenblase" zu einer Metapher für etwas, das zwar attraktiv ist, aber nicht zufrieden ist und nicht die Schönheit des Lebens widerspiegelt. In der Kunst hat der Barock Seifenblase immer als Symbol für Vanitas verwendet und spiegelt sowohl die Schönheit als auch die flüchtige Natur des menschlichen Lebens wider.'

In [ ]:
# Save the simplifications in the wiki eval data
wiki_eval['pivot'] = wiki_simpl_de
wiki_eval.to_csv(wiki_path, index=False)

In [17]:
tc_eval['pivot'] = tc_simpl_en
tc_eval.to_csv(tc_path, index=False)